## Visualizing BABEL labels
[BABEL](https://babel.is.tue.mpg.de/) labels mocap sequences from [AMASS](https://amass.is.tue.mpg.de) with action labels. 
A single sequence in BABEL can have multiple action labels associated with it, from multiple annotators. 
Here, we present code to load data from BABEL, visualize the mocap sequence rendered as a 2D video, and view the action labels corresponding to the sequence. 

In [1]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

In [2]:
import json
from os.path import join as ospj

import numpy as np

import pprint
pp = pprint.PrettyPrinter()

from IPython.display import HTML

### Load BABEL
We assume that you have downloaded BABEL annotations from the [website](https://babel.is.tue.mpg.de/data.html) and placed the downloaded `babel_v1.0_release` folder in `data/`. The BABEL data is provided as two sets -- BABEL dense and BABEL extra. 

In [3]:
d_folder = '../data/babel_v1.0_release'  # Data folder
l_babel_dense_files = ['train', 'val', 'test']
l_babel_extra_files = ['extra_train', 'extra_val']

# BABEL Dataset 
babel = {}
for file in l_babel_dense_files:
    babel[file] = json.load(open(ospj(d_folder, file+'.json')))
    
for file in l_babel_extra_files:
    babel[file] = json.load(open(ospj(d_folder, file+'.json')))    

### View random annotation

Now, let us view an annotation data structure from the BABEL. 
The overall data structure is a dictionary, with a unique sequence ID as key and the annotation as value.  

In [4]:
def get_random_babel_ann():
    '''Get annotation from random sequence from a random file'''
    file = np.random.choice(l_babel_dense_files + l_babel_extra_files)
    seq_id = np.random.choice(list(babel[file].keys()))
    print('We are visualizing annotations for seq ID: {0} in "{1}.json"'.format(seq_id, file))
    ann = babel[file][seq_id]
    return ann, file

In [5]:
ann, _ = get_random_babel_ann()
pp.pprint(ann)

We are visualizing annotations for seq ID: 3312 in "test.json"
{'babel_sid': 3312,
 'dur': 76.73,
 'feat_p': 'CMU/CMU/86/86_08_poses.npz',
 'frame_ann': {'anntr_id': 'c6065e9c-1652-46df-a45f-fe8b8158428f',
               'babel_lid': 'a642048f-7fa9-402f-a4c1-d7e9e7f696d1',
               'labels': [{'act_cat': None,
                           'end_t': 68.093,
                           'proc_label': None,
                           'raw_label': None,
                           'seg_id': 'ad703788-bd17-42d4-854b-2b64cb58ee16',
                           'start_t': 59.51},
                          {'act_cat': None,
                           'end_t': 32.82,
                           'proc_label': None,
                           'raw_label': None,
                           'seg_id': '1785aeca-53ce-4a33-a249-8a5d3466ea95',
                           'start_t': 27.445},
                          {'act_cat': None,
                           'end_t': 52.426,
                           'pr

Note that the action labels from `test.json` are not available publicly. 
Also note that the internal data structures of BABEL dense and BABEL extra differ slightly. 
For a detailed description of the annotation, see [BABEL's data page](https://babel.is.tue.mpg.de/data.html).

### Visualize a mocap seq. and its action labels 

In [6]:
def get_vid_html(url):
    '''Helper code to embed a URL in a notebook'''
    html_code = '<div align="middle"><video width="80%" controls>'
    html_code += f'<source src="{url}" type="video/mp4">'
    html_code += '</video></div>'
    return html_code

In [7]:
def get_labels(ann, file):
    # Get sequence labels and frame labels if they exist
    seq_l, frame_l = None, None
    if 'extra' not in file:
        if ann['seq_ann'] is not None:
            seq_l = [seg['raw_label'] for seg in ann['seq_ann']['labels']]
        if ann['frame_ann'] is not None:
            frame_l = [(seg['raw_label'], seg['start_t'], seg['end_t']) for seg in ann['frame_ann']['labels']]
    else:
        # Load labels from 1st annotator (random) if there are multiple annotators
        if ann['seq_anns'] is not None:
            seq_l = [seg['raw_label'] for seg in ann['seq_anns'][0]['labels']]
        if ann['frame_anns'] is not None:
            frame_l = [(seg['raw_label'], seg['start_t'], seg['end_t']) for seg in ann['frame_anns'][0]['labels']]
    return seq_l, frame_l

#### Visualize a random mocap and its annotation from BABEL, by running the cell below. 

In [8]:
ann, file = get_random_babel_ann()
seq_l, frame_l = get_labels(ann, file)
print('Sequence labels: ', seq_l)
print('Frame labels: (action label, start time, end time)')
pp.pprint(frame_l)       
HTML(get_vid_html(ann['url']))

We are visualizing annotations for seq ID: 7536 in "train.json"
Sequence labels:  ['pace and shake hand']
Frame labels: (action label, start time, end time)
[('walk', 0, 2.106),
 ('transition', 2.106, 2.845),
 ('make a knocking gesture', 2.845, 3.507),
 ('transition', 3.466, 4.6),
 ('turn around', 4.519, 5.519),
 ('walk back', 5.424, 7.734)]


- If you are interested in loading the mocap sequence in 3D, please refer to the tutorials in [AMASS](https://github.com/nghorbani/amass/tree/master/notebooks)